<a href="https://colab.research.google.com/github/avner8943/bootcamp/blob/master/radvess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from zipfile import ZipFile as zp
import cv2
import os
import numpy as np
import shutil
from glob import glob
import pickle
from tqdm import tqdm
from skimage.io import imread_collection


face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def save_face(im, count,vid_dir):

  gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

  faces = face_cascade.detectMultiScale(gray, 1.1, 4)

  x,y,w,h = faces[0]

  face_crop = gray[y:y+h,x:x+w]

  face_crop = cv2.resize(face_crop, (128,128), interpolation = cv2.INTER_AREA)

  cv2.imwrite(vid_dir + "/frame_{d}.jpg".format(d = count), face_crop)


####################################


def load_data(save_pickle = 1):



  for zip_f in glob('*.zip'):

      print(zip_f)

      actor_dir = zip_f[:-4]

      if os.path.isdir(actor_dir):
        shutil.rmtree(actor_dir)

      os.mkdir(actor_dir)

      with zp('/content/'+zip_f) as myzip:
        

        for f in myzip.infolist()[1:]:

          print(f)
          
          #vid_dir = os.path.join( actor_dir, f.filename[:-4])

          vid_dir =  f.filename[:-4]

          os.mkdir(vid_dir )

          ifile = myzip.extract(f)
          
          vidcap = cv2.VideoCapture(ifile)
             
          n_frames = np.floor((vidcap.get(cv2.CAP_PROP_FRAME_COUNT))).astype(np.int)

          frames_list = list(np.linspace(0,n_frames,60, dtype=np.int))


          success,image = vidcap.read()
          count = 0
          while success:
            if count in frames_list:
                
                save_face(image, count,vid_dir)

                if count % 10 == 0:

                  print("zip: ", zip_f, '/',len(glob('*.zip')), ", vid: ", vid_dir,'/',len(myzip.infolist()[1:]), ", frame: " , count, '/', n_frames)
                     
            success,image = vidcap.read()
            count += 1
          os.remove(ifile)
  
  
  if save_pickle == 1:
    print("picle start")
    images2pickle()

##############################################

def images2pickle(path=None):

  print("for real")

  dirs = glob("/content/*/")

  x = []
  y = []

  for dir in dirs:
    if dir.find('Actor') == -1: continue

    sub_dirs = glob(os.path.join(dir,'*/'))

    for sub_dir in sub_dirs:

      images = np.array(imread_collection(os.path.join(sub_dir, '*.jpg')))

      x.append(images)

      tag = sub_dir.split('/')[3][7]

      y.append(tag)



  x = np.array(x)

  y = np.array(y)

  with open('radvess_data_for_train.pickle', 'wb') as output:
    pickle.dump(x, output)

  with open('radvess_tags_for_train.pickle', 'wb') as output:
    pickle.dump(y, output)


####################################################

def load_pickle(path):

  with open(path, 'rb') as data:
    return pickle.load(data)




In [ ]:
load_data()



In [30]:
images2pickle()

In [9]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

x = load_pickle('radvess_data_for_train.pickle')

y = load_pickle('radvess_tags_for_train.pickle')

x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=42)

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    )

datagen.fit(x_train)

model.fit(datagen.flow(x_train, y_train, batch_size=32),
          steps_per_epoch=len(x_train) / 32, epochs=epochs)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:947: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (9, 59, 128, 128) (128 channels).
  ' channels).')


NameError: ignored